In [ ]:
!pip install -q fastapi uvicorn pyngrok nest-asyncio transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [ ]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
HF_TOKEN = userdata.get("HF_TOKEN")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

query_gen_model_name = "rajiv8197/codet5p_sql_finetuned"
query_gen_tokenizer = AutoTokenizer.from_pretrained(query_gen_model_name, token=HF_TOKEN)
query_gen_model = AutoModelForSeq2SeqLM.from_pretrained(query_gen_model_name, token=HF_TOKEN)

query_generator = pipeline("text2text-generation", model=query_gen_model, tokenizer=query_gen_tokenizer)


tokenizer_config.json:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/511k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.29M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

intent_gen_model_name = "microsoft/phi-1_5"
intent_gen_tokenizer = AutoTokenizer.from_pretrained(intent_gen_model_name)
intent_gen_model = AutoModelForCausalLM.from_pretrained(intent_gen_model_name, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)

llm = pipeline(
    "text-generation",
    model=intent_gen_model,
    tokenizer=intent_gen_tokenizer,
    do_sample=False,
    temperature=0.0,
    max_new_tokens=300
)


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def generate_intents(table_name, columns, data_types, llm=llm):
    few_shot_prompt = """
You're a helpful assistant. You are given the name of a SQL table and a list of columns with their types.
Your task is to describe the likely purpose of this table in one sentence, and then describe what each listed column likely represents.

Only describe the columns listed. Do not add or describe any columns that are not explicitly given.

Example 1:
Table: inventory
Columns:
- item_id (INTEGER)
- item_name (TEXT)
- quantity (INTEGER)
- restock_date (DATE)

Table Purpose:
Tracks the stock levels and restocking schedules of items in a warehouse.

Column Descriptions:
- item_id: Unique identifier for each inventory item.
- item_name: Name or label of the item.
- quantity: Number of units currently in stock.
- restock_date: Date when the item is expected to be restocked.

---

Example 2:
Table: employee_attendance
Columns:
- employee_id (INTEGER)
- date (DATE)
- status (TEXT)

Table Purpose:
Logs the daily attendance status of company employees.

Column Descriptions:
- employee_id: Unique identifier for the employee.
- date: The calendar date of the attendance record.
- status: Attendance status for the day (e.g., Present, Absent, Sick).

---
""".strip()

    column_lines = "\n".join(
        [f"- {col} ({dtype})" for col, dtype in zip(columns, data_types)]
    )

    prompt = (
        f"{few_shot_prompt}\n\n"
        f"Table: {table_name}\n"
        f"Columns:\n{column_lines}\n\n"
        f"Table Purpose:\n"
    )

    result = llm(prompt, return_full_text=False)[0]["generated_text"]
    generated = result.strip()

    for stop_token in ["```", "2. Write a", "CREATE TABLE", "# Solution"]:
        if stop_token in generated:
            generated = generated.split(stop_token)[0].strip()

    table_intent = ""
    column_intents = {}
    expected_columns = set([col.lower() for col in columns])

    if "Column Descriptions:" in generated:
        table_intent, column_block = generated.split("Column Descriptions:", 1)
        table_intent = table_intent.strip()

        for line in column_block.strip().splitlines():
            if line.startswith("-") and ":" in line:
                try:
                    col, intent = line[1:].split(":", 1)
                    col = col.strip().lower()
                    if col in expected_columns:
                        column_intents[col] = intent.strip()
                except ValueError:
                    continue
    else:
        table_intent = generated

    return table_intent, column_intents

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

app = FastAPI()

class IntentRequest(BaseModel):
    table_name: str
    columns: List[str]
    data_types: List[str]

class QueryRequest(BaseModel):
    prompt: str

@app.post("/generate_query")
def generate_sql(req: QueryRequest):
    result = query_generator(req.prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
    return {"sql_query": result.strip()}


@app.post("/generate_intents")
def get_intents(req: IntentRequest):
    table_intent, column_intents = generate_intents(req.table_name, req.columns, req.data_types)
    return {
        "table_intent": table_intent,
        "column_intents": column_intents
    }


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8000)
print("Your public API endpoint:", public_url)

uvicorn.run(app, port=8000)


INFO:     Started server process [533]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Your public API endpoint: NgrokTunnel: "https://e33d-34-125-229-170.ngrok-free.app" -> "http://localhost:8000"


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_intents HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
INFO:     2601:19b:784:b8e0:78fa:ce4f:d3a0:2833:0 - "POST /generate_query HTTP/1.1" 200 OK
